## Testing notebook for fitting the model of _Brunton et. al. (2013)_

In [ ]:
using Distributed

In [ ]:
# add additional processors
addprocs(44);

In [ ]:
using pulse_input_DDM

## Recover generative parameters

Simulate clicks and choices from a model, then check that generative parameters can be recovered.

In [ ]:
#parameters of the latent model
pz = Dict("generative" => vcat(1.,13.,-0.5,10.,1.,0.4,0.02), 
    "name" => vcat("σ_i","B", "λ", "σ_a","σ_s","ϕ","τ_ϕ"),
    "fit" => vcat(trues(7)),
    "initial" => vcat(2.,15.,-5.,100.,2.,0.2,0.005),
    "lb" => [eps(), 4., -5., eps(), eps(), eps(), eps()],
    "ub" => [10., 100, 5., 800., 40., 2., 10.])

#parameters for the choice observation
pd = Dict("generative" => vcat(0.1,0.1), 
    "name" => vcat("bias","lapse"), "fit" => trues(2), 
    "initial" => vcat(0.,0.5));

In [ ]:
#generate a synthetic data set of clicks and choices
ntrials = Int(2e4)
data = sample_inputs_and_choices(pz["generative"], pd["generative"], ntrials);

In [ ]:
#bin the click inputs at 10 ms
data = bin_clicks!(data;dt=1e-2);

In [ ]:
#compute the likelihood of the data, given the generative parameters
compute_LL(pz["generative"], pd["generative"], data)

In [ ]:
#find the ML parameters with gradient descent
@time pz, pd, = optimize_model(pz, pd, data);

In [ ]:
#compute the Hessian of the LL landscape, to compute confidence intervals on the parameters
pz, pd = compute_H_CI!(pz, pd, data);

In [ ]:
using DataFrames
show(DataFrame(pz),allcols=true)
show(DataFrame(pd),allcols=true)

In [ ]:
#simulate choices from the model, given the ML parameters
ML_data = deepcopy(data)
sample_choices_all_trials!(ML_data, pz["final"], pd["final"])

#compute the final click difference, which will dictate the correct choice
ΔLR = map((nT,L,R)->diffLR(nT,L,R,data["dt"])[end], data["nT"], data["leftbups"], data["rightbups"]);

In [ ]:
#bin the choices from the generative parameters and from the ML parameters
import Pandas: qcut
import Statistics: mean

nbins = 15;
conds,qcut_bins = qcut(ΔLR, nbins, labels=false, retbins=true);
conds = conds .+ 1;

frac_choice_ML = [mean(ML_data["pokedR"][conds .== i]) for i in 1:nbins]
frac_choice_gen = [mean(data["pokedR"][conds .== i]) for i in 1:nbins];

In [ ]:
#fit a GLM to the generative choices and the ML choices
using GLM
GLM_gen = glm(@formula(Y ~ X), DataFrame(X=ΔLR, Y = data["pokedR"]), Binomial(), LogitLink())
GLM_ML = glm(@formula(Y ~ X), DataFrame(X=ΔLR, Y = ML_data["pokedR"]), Binomial(), LogitLink());

In [ ]:
using PyPlot

fig = figure(figsize=(6,6))
ax = subplot(111)

scatter(qcut_bins[1:end-1] + diff(qcut_bins)/2, frac_choice_gen, color="red", label="generative")
scatter(qcut_bins[1:end-1] + diff(qcut_bins)/2, frac_choice_ML, color="grey", label="ML")

plot(sort(ΔLR), sort(predict(GLM_gen)), color="red")
plot(sort(ΔLR), sort(predict(GLM_ML)), color="grey")

ylabel("% poked R")
xlabel(L"\Delta{LR}")
ax[:spines]["top"][:set_color]("none") 
ax[:spines]["right"][:set_color]("none")
legend()

## fit real data

In [ ]:
# to fit real data, see Documentation for the expected structure of .mat files. Group animals into a 1D array
# Sessions for each animal are 1D arrays, and they are grouped into a 1D array (i.e. an array of arrays)

In [ ]:
path = "/path/to/mat/files"
ratnames=["P055"] 
sessids=[[17061800,18041700,16052600,16070500,16111700,18010300,18022600,17122200,
        17081200,17092600,17051900,18030500,17011800,18012200,17011500,17062500,
        17112700,18032700,16082200,18060800,18032500,17082900,17080500,17081000,
        17052300,17090800,17090900,18041500,18040500,17122000]];

In [ ]:
@eval $(Symbol("pz_$ratnames[1]")), $(Symbol("pz_$ratnames[1]")) = load_and_optimize(path, sessids, ratnames; dt=1e-2)

## saving things

In [ ]:
using JLD2
JLD2.@save "path/file.jld" pz pd